In [30]:
import glob
import json
import re
import nltk
import math
import copy


def calculer_ponderation_cosinus_tfidf(requete, index_inverse, idf_index, documents_trouves):
    
    ponderations = dict()
    index_requete = indexer_requete(requete)
    
    for terme in index_requete.keys():
        if terme in idf_index:#### Question: que signifie ce if ?
            index_requete[terme] *= idf_index[terme]
    
    for docA in documents_trouves:
        
        ponderations[docA] = calculer_sim_cosinus(index_inverse[docA], index_requete)
    
    return ponderations#### Question: Quelle structure de données est-ce ? Que contient-elle ?
   
    
def indexer_requete(requete):#### Question: qu'est-ce qu'il se passe ici ?
    
    index_requete = dict()
    
    for terme in requete.split():
        if terme not in index_requete:
            index_requete[terme] = 1
        else:
            index_requete[terme] += 1
            
    return index_requete

def calculer_sim_cosinus(docA, docB):
    termes_communs = list(set(docA).intersection(set(docB)))#### Question: à quoi sert set() ?
    
    #produit scalaire
    AB = 0
    for terme_commun in termes_communs:
        AB += docA[terme_commun] * docB[terme_commun]

    #norme A
    A_norme = 0
    for terme, freq in docA.items():
        A_norme += pow(freq, 2)
    A_norme = math.sqrt(A_norme)

    #norme B
    B_norme = 0
    for terme, freq in docB.items():
        B_norme += pow(freq, 2)
    B_norme = math.sqrt(B_norme)
    
    sim_cosinus = AB / (A_norme * B_norme)
    
    return sim_cosinus

def calculer_ponderation_cosinus(requete, index_inverse, documents_trouves):
    
    ponderations = dict()
    index_requete = indexer_requete(requete)
    
    for docA in documents_trouves:
    
        ponderations[docA] = calculer_sim_cosinus(index_inverse[docA], index_requete)
    
    return ponderations

def calculer_tfidf(index_inverse, index):#### Question: expliquer l'input et l'output de cette fonction 

    tfidf_index_inverse = copy.deepcopy(index_inverse)#### Question: deepcopy pourquoi ?
    # Calculer TF
    for nom, termes_document in index_inverse.items():
        totale_freqs = 0
        for _, freq in termes_document.items():
             totale_freqs += freq
        for terme in tfidf_index_inverse[nom].keys():
            tfidf_index_inverse[nom][terme] /= totale_freqs
    # Calculer IDF        
    totale_docs = len(index_inverse.keys())
    idf_index = copy.deepcopy(index)
    
    for terme, noms in idf_index.items():
        terme_rep = len(noms)
        idf_val = math.log10(totale_docs / terme_rep)
        idf_index[terme] = idf_val
    # Calculer TFIDF
    
    for nom, termes_document in tfidf_index_inverse.items():
        for terme in termes_document.keys():
            tfidf_index_inverse[nom][terme] *= idf_index[terme]
        
    return tfidf_index_inverse, idf_index
            
def lire_fichier(chemin):
    f = open(chemin, encoding="utf-8")
    chaine = f.read()
    f.close()
    return chaine

def decouper_mots(chaine):
    liste_mots = chaine.split()
    #liste_mots = nltk.word_tokenize(chaine, language="french")
    return liste_mots

def creer_index():

    index = {}

    for nom, texte in documents.items():
        for terme in texte.split():
            if terme not in index:
                index[terme] = set()
            index[terme].add(nom)

    index = {terme:sorted(list(noms)) for terme, noms in index.items()}#### Question: que signifie sorted ? 
    return index

def creer_index_inverse():

    index_inverse = {}

    for nom, texte in documents.items():
        index_inverse[nom] = dict()
        for terme in texte.split():
            if terme not in index_inverse[nom]:
                index_inverse[nom][terme] = 1
            else:
                index_inverse[nom][terme] += 1

    return index_inverse

def requeter_documets(requete, index):
    documents_trouves = list()
    requete_mots = requete.split() # découper la requête en mots
    
    # chercher pour chaque mot les documents où il apparaît
    for mot in requete_mots: 
        if mot in index:
            for document in index[mot]:
                if document not in documents_trouves:
                    documents_trouves.append(document)

    return documents_trouves

def afficher_contextes(chaine, terme, taille_contexte=30):
    match = re.search(terme, chaine)
    contexts = []
    while match is not None:
        gauche = max(match.start()-taille_contexte-1, 0)
        droite = match.end()+taille_contexte+1
        contexts.append(chaine[gauche:droite])
        chaine = chaine[match.end():]
        match = re.search(terme, chaine)
        
    for c in contexts:
        print("...", c, "...")
    
def afficher_chemins_avec_contextes(liste_chemins_trouves, requete_mots, N):
    
    if len(liste_chemins_trouves) == 0:
        print("J'ai pas trouvé des documents :(")
        return 0
    
    if N == 0:
        N = len(dic_chemins_trouves)
    
    print("- J'ai trouvé {} documents, voici les {} premiers -".format(len(liste_chemins_trouves), N))
    for similarite, chemin in liste_chemins_trouves[:N]:
        print("-> {} - {}".format(similarite, chemin))
        
        chaine = lire_fichier(chemin)
        for mot in requete_mots:
            print(">", mot, "<")
            afficher_contextes(chaine, mot)

In [13]:
index = {}
index_inverse = {}
documents = {}

chemin_corpus = "../TD4/corpus_multi/fr/*/*"##TODO: adapter ce chemin
chemin_corpus = "corpus_multi/fr/*/*"
for chemin in glob.glob(chemin_corpus):
    documents[chemin] = lire_fichier(chemin)

In [14]:
index = creer_index()
index_inverse = creer_index_inverse()

In [15]:
print("Nombre de termes différents :", len(index.keys()))
print("Nombre de documents :", len(index_inverse.keys()))

Nombre de termes différents : 33654
Nombre de documents : 272


In [19]:
requete = "liberté humaine"
docs_trouves =requeter_documets(requete, index)
print("Nombre de documents trouvés :", len(docs_trouves))

requete = "sensibilisation minorités"
docs_trouves =requeter_documets(requete, index)
print("Nombre de documents trouvés :", len(docs_trouves))

requete = "les travailleurs en difficulté"
docs_trouves =requeter_documets(requete, index)
print("Nombre de documents trouvés :", len(docs_trouves))

requete = "travailleurs difficulté"
docs_trouves =requeter_documets(requete, index)
print("Nombre de documents trouvés :", len(docs_trouves))

Nombre de documents trouvés : 23
Nombre de documents trouvés : 9
Nombre de documents trouvés : 272
Nombre de documents trouvés : 20


In [20]:
docs_trouves_pond = calculer_ponderation_cosinus(requete, index_inverse, docs_trouves)

In [21]:
docs_trouves_pond_liste = [[sim, chemin] for chemin, sim in docs_trouves_pond.items()]
docs_trouves_pond_liste = sorted(docs_trouves_pond_liste, reverse=True)
for sim, chemin in docs_trouves_pond_liste:
    print(sim, chemin.split("/")[-1])

0.06592212604335733 2009-06-03_celex_IP-09-863.fr.html
0.05564676256552816 2009-11-23_celex_IP-09-1803.fr.html
0.024613326308873328 2009-07-24_celex_IP-09-1187.fr.html
0.019553208238583517 2009-02-20_celex_IP-09-300.fr.html
0.017508866108269325 2009-11-20_celex_IP-09-1779.fr.html
0.017353946638622414 2009-02-25_celex_IP-09-317.fr.html
0.013215217891996237 2009-01-30_celex_IP-09-191.fr.html
0.01297279861366662 2009-06-18_celex_IP-09-948.fr.html
0.010282538520967822 2009-06-03_celex_IP-09-859.fr.html
0.008755775243955372 2009-07-02_celex_IP-09-1070.fr.html
0.007868407560830735 2009-03-30_celex_IP-09-501.fr.html
0.007324877167417854 2009-12-15_celex_IP-09-1926.fr.html
0.007095589971212615 2009-11-30_celex_IP-09-1842.fr.html
0.006783754721229937 2009-06-30_celex_IP-09-1060.fr.html
0.005947746617497386 2009-05-15_celex_IP-09-792.fr.html
0.005882760066687646 2009-03-03_celex_IP-09-341.fr.html
0.0056388387443460095 2009-03-03_celex_IP-09-344.fr.html
0.005524946201098299 2009-11-24_celex_IP-09

In [22]:
tfidf_index_inverse, idf_index = calculer_tfidf(index_inverse, index)

In [23]:
tfidf_docs_trouves_pond = calculer_ponderation_cosinus_tfidf(requete, tfidf_index_inverse, idf_index, docs_trouves)

In [24]:
tfidf_docs_trouves_pond_liste = [[sim, chemin] for chemin, sim in tfidf_docs_trouves_pond.items()]
tfidf_docs_trouves_pond_liste = sorted(tfidf_docs_trouves_pond_liste, reverse=True)
for sim, chemin in tfidf_docs_trouves_pond_liste:
    print(sim, chemin.split("/")[-1])

0.11337707987276982 2009-06-03_celex_IP-09-863.fr.html
0.10686024340415225 2009-11-23_celex_IP-09-1803.fr.html
0.05094431892573804 2009-07-24_celex_IP-09-1187.fr.html
0.04610680476934821 2009-02-20_celex_IP-09-300.fr.html
0.041947323739580114 2009-02-25_celex_IP-09-317.fr.html
0.04106986135007572 2009-03-30_celex_IP-09-501.fr.html
0.03777025795506586 2009-01-30_celex_IP-09-191.fr.html
0.026251816334833644 2009-06-03_celex_IP-09-859.fr.html
0.02346617617035748 2009-11-20_celex_IP-09-1779.fr.html
0.017627747511676815 2009-12-15_celex_IP-09-1926.fr.html
0.017148011742486074 2009-07-02_celex_IP-09-1070.fr.html
0.016584156668976734 2009-06-18_celex_IP-09-948.fr.html
0.014561629717160686 2009-11-24_celex_IP-09-1807.fr.html
0.013938887009736441 2009-11-30_celex_IP-09-1842.fr.html
0.013168700443911486 2009-05-15_celex_IP-09-792.fr.html
0.011329597147264931 2009-03-03_celex_IP-09-344.fr.html
0.011022361357965617 2009-06-30_celex_IP-09-1060.fr.html
0.010575889882926716 2009-03-03_celex_IP-09-341

In [25]:
requete_mots = requete.split()
print(requete)
afficher_chemins_avec_contextes(tfidf_docs_trouves_pond_liste, requete_mots, 5)

travailleurs difficulté
- J'ai trouvé 20 documents, voici les 5 premiers -
-> 0.11337707987276982 - corpus_multi/fr/appr/2009-06-03_celex_IP-09-863.fr.html
> travailleurs <
... ion et les aides en faveur des travailleurs défavorisés ou handicapés  La  ...
... ation et d'aides en faveur des travailleurs défavorisés ou handicapés port ...
... on ou d'une aide en faveur des travailleurs défavorisés ou handicapés. Ils ...
...  la formation et à l'emploi de travailleurs handicapés ou défavorisés, san ...
... ise et par an pour l'emploi de travailleurs défavorisés et à 10 millions € ...
... ise et par an pour l'emploi de travailleurs handicapés.  Les documents d'o ...
> difficulté <
-> 0.10686024340415225 - corpus_multi/fr/appr/2009-11-23_celex_IP-09-1803.fr.html
> travailleurs <
... es, les moins qualifiés et les travailleurs temporaires qui ont été le plu ...
... chaque année, quelque 22 % des travailleurs européens changent d’emploi. C ...
... turelle fondamentale. Tous les travailleurs n

In [32]:
requete = "la solidarité"

#Frecs sans TFIDF
docs_trouves = requeter_documets(requete, index)
print("Nombre de documents trouvées :", len(docs_trouves))
docs_trouves_pond = calculer_ponderation_cosinus(requete, index_inverse, docs_trouves)
docs_trouves_pond_liste = [[sim, chemin] for chemin, sim in docs_trouves_pond.items()]
docs_trouves_pond_liste = sorted(docs_trouves_pond_liste, reverse=True)
for sim, chemin in docs_trouves_pond_liste:
    if sim > 0.0:    
        print(sim, chemin.split("/")[-1])

Nombre de documents trouvées : 272
0.39321842779383454 2009-11-17_celex_IP-09-1726.fr.html
0.38026451374359754 2009-03-03_celex_IP-09-342.fr.html
0.37822433173570263 2009-10-29_celex_IP-09-1626.fr.html
0.36465896644774715 2009-09-11_celex_IP-09-1305.fr.html
0.36094094744087163 2009-04-29_celex_IP-09-673.fr.html
0.35722813480814675 2009-06-10_celex_IP-09-893.fr.html
0.35689566054702565 2009-04-22_celex_IP-09-617.fr.html
0.3514797457833188 2009-06-09_celex_IP-09-891.fr.html
0.3442254855641126 2009-06-12_celex_IP-09-910.fr.html
0.34386411670267625 2009-01-22_celex_IP-09-112.fr.html
0.3436454497774016 2009-04-23_celex_IP-09-625.fr.html
0.3370848210798291 2009-07-24_celex_IP-09-1188.fr.html
0.3359902646802143 2009-04-27_celex_IP-09-644.fr.html
0.33351147024782035 2009-11-13_celex_IP-09-1710.fr.html
0.3330738105798536 2009-05-15_celex_IP-09-792.fr.html
0.32961063021678666 2009-06-03_celex_IP-09-863.fr.html
0.32907259085720875 2009-07-09_celex_IP-09-1109.fr.html
0.328816336544921 2009-05-28_c

In [29]:
#TFIDF
print(requete)
docs_trouves = requeter_documets(requete, index)
print("Nombre de documents trouvées :", len(docs_trouves))
tfidf_docs_trouves_pond = calculer_ponderation_cosinus_tfidf(requete, tfidf_index_inverse, idf_index, docs_trouves)
tfidf_docs_trouves_pond_liste = [[sim, chemin] for chemin, sim in tfidf_docs_trouves_pond.items()]
tfidf_docs_trouves_pond_liste = sorted(tfidf_docs_trouves_pond_liste, reverse=True)
for sim, chemin in tfidf_docs_trouves_pond_liste[:10]:
    if sim > 0.0:
        print(sim, chemin.split("/")[-1])
chemin_fichier1 = tfidf_docs_trouves_pond_liste[0][1]
print("")
print(lire_fichier(chemin_fichier1)[:200])

travailleurs difficulté
Nombre de documents trouvées : 20
0.11337707987276982 2009-06-03_celex_IP-09-863.fr.html
0.10686024340415225 2009-11-23_celex_IP-09-1803.fr.html
0.05094431892573804 2009-07-24_celex_IP-09-1187.fr.html
0.04610680476934821 2009-02-20_celex_IP-09-300.fr.html
0.041947323739580114 2009-02-25_celex_IP-09-317.fr.html
0.04106986135007572 2009-03-30_celex_IP-09-501.fr.html
0.03777025795506586 2009-01-30_celex_IP-09-191.fr.html
0.026251816334833644 2009-06-03_celex_IP-09-859.fr.html
0.02346617617035748 2009-11-20_celex_IP-09-1779.fr.html
0.017627747511676815 2009-12-15_celex_IP-09-1926.fr.html

IP/09/ 863  Bruxelles, le 3 juin 2009 Aides d'État: la Commission adopte des orientations sur les aides à la formation et les aides en faveur des travailleurs défavorisés ou handicapés  La Commission 


In [1]:
toto = {"titi":0}

tutu = toto

tutu["titi"]+=1
print(toto)
print(tutu)

{'titi': 1}
{'titi': 1}
